In [ ]:
from PIL import Image
import os
from tqdm import tqdm

In [ ]:
INPUT_BASE = 'Data/Heatmaps/ValidateGaussianNormalizedCenter'
SUBDIRS = ['Positive', 'Neutral', 'Negative']

OUTPUT_BASE = 'Data/Heatmaps/Normalized-Center-Gaussians/Validate'
SIZE = 256, 256

In [ ]:
for subdir in SUBDIRS:
    image_dir = os.path.join(INPUT_BASE, subdir)
    output_dir = os.path.join(OUTPUT_BASE, subdir)
    for root, dirs, filenames in os.walk(image_dir):
        for filename in tqdm(filenames):
            img_path = os.path.join(root, filename)
            img = Image.open(img_path)
            resized = img.resize(SIZE, Image.LANCZOS)
            resized.save(os.path.join(output_dir, filename), 'PNG')

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

In [ ]:
# Data augmentation
train_datagen = ImageDataGenerator(
          rotation_range=40, width_shift_range=0.2,
          height_shift_range=0.2, rescale=1./100,
          shear_range=0.2, zoom_range=0.2,
          horizontal_flip=True, fill_mode='nearest'
           )

validate_datagen = ImageDataGenerator(rescale=1./100)

In [ ]:
train_data_dir = 'Data/Heatmaps/Simple-Gaussians/train'
validation_data_dir = 'Data/Heatmaps/Simple-Gaussians/validation'
nb_train_samples = 1117 + 1150 + 1241
nb_validation_samples = 528 + 689 + 761
epochs = 30
batch_size = 16


In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(3, 256, 256)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
train_generator = train_datagen.flow_from_directory(
                    train_data_dir,
                    target_size=(256, 256),
                    batch_size=batch_size,
                    class_mode='categorical'
                    )

validation_generator = validate_datagen.flow_from_directory(
                            validation_data_dir,
                            target_size=(256, 256),
                            batch_size=batch_size,
                            class_mode='categorical'
                            )

In [ ]:
model.fit_generator(train_generator, 
                    steps_per_epoch=nb_train_samples // batch_size,
                    epochs=epochs,
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size
                   )